I - Initialisation </br>
Import des librairies et modules

In [1]:

import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px  # Correction ici
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder


II - Lecture et conversion des jeux de données </br>
- Csv -> DataFrame </br>
- Fusion des deux jeux de données

In [2]:

train = pd.read_csv('data/train.csv')
test = pd.read_csv('data/test.csv')
test['Survived'] = None
full = pd.concat([train, test], sort=False)


III - Traitement des données </br>
- Extraction des titres dans 'Name' et création d'une colonne 'Title' </br>
- Remplacement des données manquantes : l'âge est déterminé par médiane par groupe </br>
- Encodage des données de type 'str' en valeurs numériques utilisables par les modèles d'entrainement </br>
- Suppression des colonnes (jugées) inutiles à l'analyse : 'Name', 'Ticket', 'Cabin'

In [3]:

# Extraction titre
full['Title'] = full['Name'].str.extract(r',\s*([^\.]*)\s*\.', expand=False).str.strip()

# Remplissage des âges manquants par médiane par groupe
age_medians = full.groupby(['Title', 'Pclass'])['Age'].median()
def impute_age(row):
    if pd.isnull(row['Age']):
        return age_medians.get((row['Title'], row['Pclass']), full['Age'].median())
    return row['Age']
full['Age'] = full.apply(impute_age, axis=1)

# Encodage des variables catégorielles
le = LabelEncoder()
full['Sex'] = le.fit_transform(full['Sex'])
full['Title'] = le.fit_transform(full['Title'])
full['Embarked'] = full['Embarked'].fillna(full['Embarked'].mode()[0])
full['Embarked'] = le.fit_transform(full['Embarked'])

# Remplissage de 'Fare'
full['Fare'] = full['Fare'].fillna(full['Fare'].median())

# Suppression des colonnes inutiles
full.drop(columns=['Cabin', 'Ticket', 'Name'], inplace=True)


IV - Séparation des données en deux groupes </br>
- train_cleaned = contient uniquement les lignes avec une valeur connue pour 'Survived'(utilisé pour entrainer le modèle)
- test_cleaned = contient les lignes sans valeur de 'Survived' (utilisé pour faire des prédictions)

In [ ]:
train_cleaned = full[full['Survived'].notnull()]
test_cleaned = full[full['Survived'].isnull()]

V - Sélection des features et de la target

In [5]:

features = ['Age', 'Sex', 'Pclass', 'Title', 'Parch', 'Fare', 'SibSp']
X = train_cleaned[features]
y = train_cleaned['Survived'].astype(int)


VI - Division des données en 2 groupes : 80% pour l'entrainement, 20% pour le test

In [6]:

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=1)


VII - Entrainement du modèle

In [17]:

model = RandomForestClassifier(
    n_estimators=100,
    max_depth=None,
    random_state=1)
model.fit(X_train, y_train)
y_pred = model.predict(X_test)


VIII - Evaluation </br>


In [18]:

print("Accuracy:", accuracy_score(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))
print(classification_report(y_test, y_pred))


Accuracy: 0.7988826815642458
[[96 10]
 [26 47]]
              precision    recall  f1-score   support

           0       0.79      0.91      0.84       106
           1       0.82      0.64      0.72        73

    accuracy                           0.80       179
   macro avg       0.81      0.77      0.78       179
weighted avg       0.80      0.80      0.79       179



VIII.bis - Interprétation des résultats

- L'accuracy (précision) est estimée à 79~80%


- La matrice de confusion : 96 Vrais Positif (prédits vivants et vivants) / 10 Faux Positifs (prédits vivants et morts) / 26 Faux Négatifs (prédits morts et vivants) / 47 Vrais Négatifs (prédits morts et morts)

Classification report

- Classe 0 (mort) </br>
1- Précision : 79% de bonnes prédictions sur les passagers effectivement mortes </br>
2- Recall : Sur l'ensemble des vrais morts, 91% ont bien été prédits </br>
3- F1-score : Moyenne entre precision et recall (84%)

- Classe 1 (vivant) </br>
1- Précision : 82% de bonnes prédictions sur les passagers effectivement vivants </br>
2- Recall : Sur l'ensemble des vrais vivants, 64% ont bien été prédits </br>
3- F1-score : Moyenne entre precision et recall (72%)

Synthèse </br>
Le modèle a une précision de 80%, ce qui signifie que sur 10 passagers, 8 ont reçus une prédiction en adéquation avec les données réelles. </br>
On constate avec la matrice de confusion que le modèle est efficace pour prédire les morts (96/106) mais a des difficultés à prédire les survivants (47/73) avec en faux négatif 26 individus. </br>
Le rapport de classification indique un rappel (recall) pour les morts élévé (0.91) mais plus faible pour les survivants (0.64). Ce qui peut se traduire par une tendance à prédire plus souvent la mort d'un passager. </br>
En bref, le modèle est efficace en ce qui concerne la prédiction des morts mais doit être amélioré pour prédire les survivants.


IX - Exportation des résultats

In [10]:

X_test_final = test_cleaned[features]


y_pred_final = model.predict(X_test_final)
submission = test_cleaned[['PassengerId']].copy()
submission['Survived'] = y_pred_final.astype(int)
submission.to_csv('submission.csv', index=False)
